In [7]:
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone

API_TOKEN = "KuQ4Rt1ypOCvXfcm3cZXdPhOUlbuOrpHBgJkFm1MWTvtRR8TLhgEI02hjDxz"
BASE_URL = "https://api.sportmonks.com/v3/football"
BOOKMAKER_URL = "https://api.sportmonks.com/v3/odds/bookmakers"


# -------------------------
# Fixtures (next 12 hours)
# -------------------------
def fetch_fixtures_next_12h() -> pd.DataFrame:
    now_utc = datetime.now(timezone.utc)
    end_utc = now_utc + timedelta(hours=48)

    start_date = now_utc.date()
    end_date = (now_utc + timedelta(days=1)).date()
    endpoint = f"/fixtures/between/{start_date:%Y-%m-%d}/{end_date:%Y-%m-%d}"

    session = requests.Session()
    fixtures = []
    page = 1

    while True:
        r = session.get(
            f"{BASE_URL}{endpoint}",
            params={"api_token": API_TOKEN, "page": page},
            timeout=30,
        )
        r.raise_for_status()
        payload = r.json()

        fixtures.extend(payload.get("data") or [])

        pagination = ((payload.get("meta") or {}).get("pagination")) or {}
        if pagination.get("has_more") is False:
            break
        if pagination.get("current_page") == pagination.get("total_pages"):
            break

        page += 1

    df = pd.DataFrame(
        {
            "FixtureID": [f.get("id") for f in fixtures],
            "Fixture": [f.get("name") for f in fixtures],
            "StartingAt": [f.get("starting_at") for f in fixtures],
        }
    )

    if df.empty:
        return df

    df["StartingAt"] = pd.to_datetime(df["StartingAt"], utc=True, errors="coerce")
    df = df[df["StartingAt"].between(now_utc, end_utc)].reset_index(drop=True)
    return df


# -------------------------
# Bookmaker ID -> Name lookup
# -------------------------
def _get_bookmaker_name(bookmaker_id, session, cache):
    if bookmaker_id is None:
        return None
    if bookmaker_id in cache:
        return cache[bookmaker_id]

    r = session.get(
        f"{BOOKMAKER_URL}/{bookmaker_id}",
        params={"api_token": API_TOKEN},
        timeout=30,
    )
    if r.status_code != 200:
        return None

    name = (r.json().get("data") or {}).get("name")
    if name:
        cache[bookmaker_id] = name
    return name


# -------------------------
# Odds (extract ALL fields, nothing else)
# -------------------------
def fetch_odds_prematch_for_fixture(
    fixture_id: int,
    session: requests.Session,
    bookmaker_cache: dict,
) -> list[dict]:

    endpoint = f"/odds/pre-match/fixtures/{fixture_id}"

    r = session.get(
        f"{BASE_URL}{endpoint}",
        params={"api_token": API_TOKEN},
        timeout=30,
    )
    if r.status_code != 200:
        return []

    odds = r.json().get("data") or []
    rows = []

    for odd in odds:
        bookmaker_id = odd.get("bookmaker_id")
        bookmaker_name = _get_bookmaker_name(bookmaker_id, session, bookmaker_cache)

        rows.append(
            {
                # IDs
                "OddID": odd.get("id"),
                "FixtureID": odd.get("fixture_id"),
                "MarketID": odd.get("market_id"),
                "BookmakerID": bookmaker_id,
                "Bookmaker": bookmaker_name,

                # Market / selection
                "Market": odd.get("market_description"),
                "Label": odd.get("label"),
                "Name": odd.get("name"),

                # Odds formats
                "Value": odd.get("value"),
                "Probability": odd.get("probability"),
                "DP3": odd.get("dp3"),
                "Fractional": odd.get("fractional"),
                "American": odd.get("american"),

                # Status
                "Winning": odd.get("winning"),
                "Stopped": odd.get("stopped"),

                # Lines
                "Total": odd.get("total"),
                "Handicap": odd.get("handicap"),

                # Extra
                "Participants": odd.get("participants"),
                "LatestBookmakerUpdate": odd.get("latest_bookmaker_update"),
            }
        )

    return rows


def fetch_odds_next_12h() -> pd.DataFrame:
    fixtures_df = fetch_fixtures_next_12h()
    if fixtures_df.empty:
        return pd.DataFrame()

    session = requests.Session()
    bookmaker_cache = {}
    rows = []

    for fid in fixtures_df["FixtureID"]:
        rows.extend(fetch_odds_prematch_for_fixture(fid, session, bookmaker_cache))

    df = pd.DataFrame(rows)
    if df.empty:
        return df

    df = df.merge(fixtures_df, on="FixtureID", how="left")
    return df


# -------------------------
# Example usage
# -------------------------
df = fetch_odds_next_12h()
df.head(20)

,OddID,FixtureID,MarketID,BookmakerID,Bookmaker,Market,Label,Name,Value,Probability,...,Fractional,American,Winning,Stopped,Total,Handicap,Participants,LatestBookmakerUpdate,Fixture,StartingAt
0,210917527889,19425061,126,2,bet365,Winning Margin,1,3,17.00,5.88%,...,17,1600,False,False,None,,None,2026-01-06 20:31:27,Bologna vs Atalanta,2026-01-07 17:30:00+00:00
1,210906230548,19425061,57,2,bet365,Correct Score,Draw,4-4,201.00,0.5%,...,201,20000,False,False,None,None,None,2026-01-06 20:31:27,Bologna vs Atalanta,2026-01-07 17:30:00+00:00
2,210921900257,19425061,90,2,bet365,Goalscorers,First,Honest Ahanor,29.00,3.45%,...,29,2800,False,False,None,None,None,2026-01-06 20:31:27,Bologna vs Atalanta,2026-01-07 17:30:00+00:00
3,211855537685,19425061,334,2,bet365,Player Shots On Target,0.5,Nikola Moro,3.50,28.57%,...,7/2,250,False,False,None,None,None,2026-01-06 20:31:27,Bologna vs Atalanta,2026-01-07 17:30:00+00:00
4,210909649489,19425061,92,2,bet365,Team Goalscorer,First,Henry Camara,4.00,25%,...,4,300,False,False,None,None,None,2026-01-06 20:31:27,Bologna vs Atalanta,2026-01-07 17:30:00+00:00
5,210917529021,19425061,100,2,bet365,Multi Scorers,3 or More,Henry Camara,67.00,1.49%,...,67,6600,False,False,None,None,None,2026-01-06 20:31:27,Bologna vs Atalanta,2026-01-07 17:30:00+00:00
6,211865106012,19425061,337,2,bet365,Player Headed Shots on Target,2.5,Ciro Immobile,51.00,1.96%,...,51,5000,False,False,None,None,None,2026-01-06 20:31:27,Bologna vs Atalanta,2026-01-07 17:30:00+00:00
7,210909649592,19425061,251,2,bet365,1st Goal Scorer,First,Bodin Tomasevic,21.00,4.76%,...,21,2000,False,False,None,None,None,2026-01-06 20:31:27,Bologna vs Atalanta,2026-01-07 17:30:00+00:00
8,211295450506,19425061,31,3,188Bet,1st Half Winner,Home,Home,3.35,29.85%,...,57/17,235,False,False,None,None,None,2026-01-04 14:18:20,Bologna vs Atalanta,2026-01-07 17:30:00+00:00
9,211295450516,19425061,31,35,1xbet,1st Half Winner,Draw,Draw,2.04,49.02%,...,49/24,104,False,False,None,None,None,2026-01-06 12:01:21,Bologna vs Atalanta,2026-01-07 17:30:00+00:00


In [8]:
# show all unique markets based on marketid

markets = (
    df.groupby("MarketID", dropna=False)["Market"]
      .unique()
      .reset_index()
      .sort_values("MarketID")
      .reset_index(drop=True)
)

markets

,MarketID,Market
0,1,"[Full Time Result, Fulltime Result, Match Winner]"
1,2,[Double Chance]
2,6,[Asian Handicap]
3,7,[Goal Line]
4,10,[Draw No Bet]
...,...,...
135,333,[Player to Score or Assist]
136,334,[Player Shots On Target]
137,335,[Player Shots on Target Outside Box]
138,336,[Player Shots]


In [11]:
market_name_counts = (
    df.groupby("MarketID")["Market"]
      .nunique()
      .reset_index(name="DistinctMarketNames")
)

market_name_counts[market_name_counts["DistinctMarketNames"] > 1] \
    .sort_values("DistinctMarketNames", ascending=False)

,MarketID,DistinctMarketNames
0,1,3
78,93,2
34,45,2
36,47,2
42,53,2
48,62,2
53,67,2
68,82,2
81,97,2
5,11,2


In [9]:
market_labels = (
    df.groupby("MarketID", dropna=False)["Label"]
      .unique()
      .reset_index()
      .sort_values("MarketID")
      .reset_index(drop=True)
)

market_labels


,MarketID,Label
0,1,"[Home, Away, Draw]"
1,2,"[Home/Draw, Home/Away, Draw/Away, Bologna or D..."
2,6,"[Home, Away, 1, 2]"
3,7,"[Under, Over]"
4,10,"[1, 2]"
...,...,...
135,333,[To Score or Assist]
136,334,"[0.5, 1.5, 2.5, 3.5, 4.5, 5.5]"
137,335,"[0.5, 1.5, 2.5, 3.5]"
138,336,"[3.5, 0.5, 1.5, 4.5, 5.5, 6.5, 2.5, 7.5, 8.5, ..."
